In [1]:
from ast import literal_eval
import polars as pl

from pathlib import Path

base_dir = Path("./data")
base_dir.mkdir(exist_ok=True)

In [2]:
# Convert to Parqeet
prachathai = pl.read_csv('prachathai-67k.csv', schema_overrides={
    "date": pl.Datetime(),
}).with_columns(
    # Parquet already support complex schemas
    # So the workaround with `literal_eval` for deserializing is no longer required.
    pl.col("labels").map_elements(literal_eval, return_dtype=pl.List(pl.Utf8)),
)

print("Data shape:", prachathai.shape)
print("Data schema:")
display(prachathai.schema)

prachathai.head()

Data shape: (67889, 5)
Data schema:


Schema([('url', String),
        ('date', Datetime(time_unit='us', time_zone=None)),
        ('title', String),
        ('body_text', String),
        ('labels', List(String))])

url,date,title,body_text,labels
str,datetime[μs],str,str,list[str]
"""https://prachatai.com/print/42""",2004-08-24 14:31:00,"""นักวิจัยหนุน ""แม้ว"" เปิด ""จ…","""ประชาไท --- 23 ส.ค.2547 นักวิ…","[""ข่าว"", ""สิ่งแวดล้อม""]"
"""https://prachatai.com/print/41""",2004-08-24 14:22:00,"""ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ""","""ประชาไท- 23 ส.ค.2547 นักวิชา…","[""ข่าว"", ""สิ่งแวดล้อม""]"
"""https://prachatai.com/print/43""",2004-08-24 15:17:00,"""จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย""",""" นโยบายที่อนุญาตให้ปลูกร่วมกับ…","[""ข่าว"", ""สิ่งแวดล้อม""]"
"""https://prachatai.com/print/45""",2004-08-24 15:58:00,"""ฟองสบู่การเมืองแตก ทักษิณหมดกึ…","""ประชาไท -- 23 ส.ค. 47 ขาประจำ…","[""ข่าว"", ""การเมือง"", … ""เสวนา""]"
"""https://prachatai.com/print/47""",2004-08-24 16:10:00,"""กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง""","""ประชาไท-23 ส.ค.47 คณะกรรมการอ…","[""ข่าว"", ""สิ่งแวดล้อม""]"


In [3]:
# Write original dataset
original_dir = base_dir / "original"
original_dir.mkdir(exist_ok=True)

prachathai.write_parquet(original_dir / "train.parquet")

In [4]:
tags = ['การเมือง','สิทธิมนุษยชน','คุณภาพชีวิต','ต่างประเทศ','สังคม',
                  'สิ่งแวดล้อม','เศรษฐกิจ','วัฒนธรรม','แรงงาน','ความมั่นคง',
                  'ไอซีที','การศึกษา']
tags_en = [
    'politics',
    'human_rights',
    'quality_of_life',
    'international',
    'social',
    'environment',
    'economics',
    'culture',
    'labor',
    'national_security',
    'ict',
    'education',
]

In [5]:
# Pick labels and encode one hot
prachathai_encoded = prachathai.with_columns([
    pl.col('labels').list.contains(tag).cast(pl.Int8).alias(tag_en)
    for tag, tag_en in zip(tags, tags_en)
]).drop("labels")

print("Encoded Data shape:", prachathai_encoded.shape)
print("Encoded Data schema:")
display(prachathai_encoded.schema)

prachathai_encoded.head()

Encoded Data shape: (67889, 16)
Encoded Data schema:


Schema([('url', String),
        ('date', Datetime(time_unit='us', time_zone=None)),
        ('title', String),
        ('body_text', String),
        ('politics', Int8),
        ('human_rights', Int8),
        ('quality_of_life', Int8),
        ('international', Int8),
        ('social', Int8),
        ('environment', Int8),
        ('economics', Int8),
        ('culture', Int8),
        ('labor', Int8),
        ('national_security', Int8),
        ('ict', Int8),
        ('education', Int8)])

url,date,title,body_text,politics,human_rights,quality_of_life,international,social,environment,economics,culture,labor,national_security,ict,education
str,datetime[μs],str,str,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
"""https://prachatai.com/print/42""",2004-08-24 14:31:00,"""นักวิจัยหนุน ""แม้ว"" เปิด ""จ…","""ประชาไท --- 23 ส.ค.2547 นักวิ…",0,0,0,0,0,1,0,0,0,0,0,0
"""https://prachatai.com/print/41""",2004-08-24 14:22:00,"""ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ""","""ประชาไท- 23 ส.ค.2547 นักวิชา…",0,0,0,0,0,1,0,0,0,0,0,0
"""https://prachatai.com/print/43""",2004-08-24 15:17:00,"""จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย""",""" นโยบายที่อนุญาตให้ปลูกร่วมกับ…",0,0,0,0,0,1,0,0,0,0,0,0
"""https://prachatai.com/print/45""",2004-08-24 15:58:00,"""ฟองสบู่การเมืองแตก ทักษิณหมดกึ…","""ประชาไท -- 23 ส.ค. 47 ขาประจำ…",1,0,0,0,0,0,0,0,0,0,0,0
"""https://prachatai.com/print/47""",2004-08-24 16:10:00,"""กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง""","""ประชาไท-23 ส.ค.47 คณะกรรมการอ…",0,0,0,0,0,1,0,0,0,0,0,0


In [6]:
# Write encoded dataset
labeled_dir = base_dir / "labeled"
labeled_dir.mkdir(exist_ok=True)

prachathai_encoded.write_parquet(labeled_dir / "train.parquet")

In [7]:
from sklearn.model_selection import train_test_split

print("Split train/validation/test...")

train_valid, test = train_test_split(prachathai_encoded, test_size=0.1, random_state=1412)
train, valid = train_test_split(train_valid, test_size=0.11, random_state=1412)

print(train.shape, valid.shape, test.shape)

out_dir = base_dir / "default"
out_dir.mkdir(exist_ok=True)

for name, df in zip(["train", "validation", "test"], [train, valid, test]):
    print("Saving", name)
    df.write_parquet(out_dir / f"{name}.parquet")

Split train/validation/test...
(54379, 16) (6721, 16) (6789, 16)
Saving train
Saving validation
Saving test
